<a href="https://colab.research.google.com/github/WesleyKhoiVo/DeepLearning/blob/main/Lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Load MNIST dataset
# MNIST dataset has a shape of (70000, 784) meaning there are 70,000 images with 784 dimensions (784 features).
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", as_frame=False, parser='auto')

In [2]:
#Explore the first image in MNIST dataset
mnist['data'].loc[0]

(60000, 784)

In [3]:
import matplotlib.pyplot as plt
plt.imshow(mnist.data[0].reshape(28,28), cmap='gray')

In [4]:
mnist.target[0]

In [5]:
from sklearn.model_selection import train_test_split
# test_size: what proportion of original data is used for test set
train_img, test_img, train_lbl, test_lbl = train_test_split( mnist.data, mnist.target, test_size=1/7.0, random_state=0)
train_lbl[0:10]
train_img.shape

In [6]:
from tensorflow.keras.utils import to_categorical
train_onehot_lbl = to_categorical(train_lbl,num_classes=10)
test_onehot_lbl = to_categorical(test_lbl,num_classes=10)
train_onehot_lbl[0:10]

[1,  2000] loss: 1.879
[1,  4000] loss: 1.389
[1,  6000] loss: 1.190
[2,  2000] loss: 1.081
[2,  4000] loss: 0.955
[2,  6000] loss: 0.896
Finished Training


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

input_layer = Input(shape=(784,))
hidden_layer = Dense(30,activation="sigmoid")(input_layer)
dense_layer = Dense(10, activation="softmax")(hidden_layer)

model = Model(inputs=input_layer, outputs=dense_layer)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

model.fit(train_img,train_onehot_lbl,validation_data=(test_img,test_onehot_lbl), epochs= 20, batch_size=20)

tensor([8, 4, 1,  ..., 1, 3, 0])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

input_layer = Input(shape=(784,))
dense_layer1 = Dense(100,activation="sigmoid")(input_layer) 
dense_layer2 = Dense(80,activation="sigmoid")(dense_layer1)
dense_layer3 = Dense(50,activation="sigmoid")(dense_layer2)
dense_layer4 = Dense(10, activation="softmax")(dense_layer3)  

model = Model(inputs=input_layer, outputs=dense_layer4)
model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=["accuracy"])

model.summary()

#model.fit(train_img,train_onehot_lbl,validation_data=(test_img,test_onehot_lbl), epochs= 5, batch_size=10)


In [ ]:
model.predict(test_img[0:2])